## Different Classifier Models: Practicality, Advantages and Performance
 
**Data** [Sources](../Data/Sources.ipynb) | [Compositional Data](../Data/CompositionalData.ipynb) | [Lambdas](../Data/Lambdas.ipynb) **Classifiers** [Models](../Classifiers/ClassifierModels.ipynb)| [Dimensionality](../Classifiers/Dimensionality.ipynb) | [Uncertainty](../Classifiers/Uncertainty.ipynb) **Visualisation**  [Entropy](../Vis/Entropy.ipynb) | [Manifolds](../Vis/Manifolds.ipynb) **Workflows** [Workflow](../Workflow.ipynb) | [Customising](../Workflow.ipynb) **GitHub** [AEGC2019](https://github.com/morganjwilliams/aegc2019) | [pyrolite](https://github.com/morganjwilliams/pyrolite)

([Petrelli2016], [Ueki2018])

[Petrelli2016]: https://doi.org/10.1007/s00410-016-1292-2 "Petrelli, M., Perugini, D., 2016. Solving petrological problems through machine learning: the study case of tectonic discrimination using geochemical and isotopic data. Contrib Mineral Petrol 171, 81."
[Ueki2018]: https://doi.org/10.1029/2017GC007401 "Ueki, K., Hino, H., Kuwatani, T., 2018. Geochemical Discrimination and Characteristics of Magmatic Tectonic Settings: A Machine-Learning-Based Approach. Geochemistry, Geophysics, Geosystems 19, 1327–1347."

### Support Vector Classifiers

### Random Forests

### Multinomial Logistic Regressions